In [1]:
import json
import uuid
from multiprocessing import Pool
from tqdm.auto import tqdm
import random
import cv2
import os
from timeout_decorator import timeout, TimeoutError

In [2]:
with open("output_buckup.json", "r") as fp:
    data=fp.readlines()[0]

In [3]:
data = json.loads(data)
len(data)

843514

In [4]:
@timeout(600)
def _trim(item):
    size = 512
    key, values = item
    img = cv2.imread(key)
    if img is None:
        print("error loading", key)
        return
    for value in values:
        try:
            score = value["score"]
            if score<0.8:
                continue
            bbox = value["bbox"]
            min_x, min_y, max_x, max_y = bbox
            min_x = int(min_x)
            min_y = int(min_y)
            max_x = int(max_x)
            max_y = int(max_y)
            
            _size = min(max_y-min_y, max_x-min_x)
            if _size<size*0.7:
                continue
            left = random.randint(0,min(min_x, _size*2))
            right = random.randint(max_x, min(img.shape[1]-1, max_x+_size*2))
            up = random.randint(0,min(min_y, _size*2))
            down = random.randint(max_y, min(img.shape[0]-1, max_y+_size*2))
            if abs(right-left)>abs(down-up):
                delta = abs(right-left) - abs(down-up)
                assert delta>=0
                right-=delta//2
                left+=delta//2
            else:
                delta = abs(down-up) - abs(right-left)
                assert delta>=0
                up+=delta//2
                down-=delta//2
            img_trim = img.copy()[up:down, left:right]
            img_trim = cv2.resize(img_trim, (size, size))
            cv2.imwrite("faces/{0}-{1}.jpg".format(str(uuid.uuid4()), str(uuid.uuid4())), img_trim)
        except Exception as e:
            print(key)
            print(e)
    
def trim(item):
    try:
        _trim(item)
    except TimeoutError:
        pass

In [5]:
os.makedirs("faces", exist_ok=True)
for i in range(5):
    with Pool() as p:
            imap = p.imap(trim, list(data.items()))
            list(tqdm(imap, total=len(data)))

  0%|          | 0/843514 [00:00<?, ?it/s]

  0%|          | 0/843514 [00:00<?, ?it/s]

  0%|          | 0/843514 [00:00<?, ?it/s]

  0%|          | 0/843514 [00:00<?, ?it/s]

  0%|          | 0/843514 [00:00<?, ?it/s]